## Transcribing

In [ ]:
import os
import csv
from pathlib import Path
import torch
from stable_whisper import load_model

# === SETTINGS ===
root_dir = r"..."
output_csv = r"..."  # ✅ Save here
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {device}")

# === LOAD MODEL ===
model = load_model("large-v3", device=device)

# === HELPER FUNCTION ===
def transcribe_audio_files(directory):
    results = []

    # Walk through all subdirectories
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if filename.lower().endswith(".wav"):
                filepath = os.path.join(root, filename)
                print(f"🎧 Transcribing: {filepath}")

                try:
                    result = model.transcribe(filepath, language="az")
                    text = result.text.strip()  # ✅ Access object attribute, not dict

                    if text:  # Skip empty transcriptions
                        results.append([text, 1])  # Dummy label '1'
                except Exception as e:
                    print(f"⚠️ Failed to transcribe {filename}: {e}")
    return results

# === PROCESS ===
transcriptions = transcribe_audio_files(root_dir)

# === SAVE TO CSV ===
if transcriptions:
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)  # ✅ Ensure output dir exists

    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Transcription", "Label"])  # Header
        writer.writerows(transcriptions)

    print(f"✅ Transcriptions saved to {output_csv}")
else:
    print("❌ No transcriptions were generated.")

### Single-Audio Transcrabing

In [ ]:
import os
import csv
import torch
from stable_whisper import load_model

# === SETTINGS ===
audio_path = r".wav"
output_csv = r".csv"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 Using device: {device}")

# === LOAD MODEL ===
model = load_model("large-v3", device=device)

# === TRANSCRIBE SINGLE AUDIO ===
try:
    print(f"🎧 Transcribing: {audio_path}")
    result = model.transcribe(audio_path, language="az")
    text = result.text.strip()

    if text:
        # === SAVE TO CSV ===
        with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["Transcription", "Label"])  # Header
            writer.writerow([text, 1])  # Dummy label

        print(f"✅ Transcription saved to {output_csv}")
    else:
        print("⚠️ Transcription is empty.")
except Exception as e:
    print(f"❌ Error during transcription: {e}")

In [ ]:
import pandas as pd

df = pd.read_csv(r".csv")
display(df)
print()
display(df['Transcription'].iloc[0])

## Data Preprocessing & Preparation

In [ ]:
import pandas as pd

df = pd.read_csv(r".csv")
display(df)
print()
display(df['Transcription'].iloc[0])

In [ ]:
display(df['Transcription'].iloc[97])

In [ ]:
import pandas as pd

# Existing DataFrame
df_transcripts = pd.read_csv(r".csv")

# Pasting synthetic transcriptions here as a list of strings
negative_samples = [
    "alo salam mən xaricə getmək üçün məhküm olmamağım barədə sənəd lazım olduğunu dedilər onu asan xidmətdən ala bilərəmmi salam xanım bu məsələyə biz baxmırıq deyəsən başqa yerdə verirlər belə sənədləri məncə siz notariusa yaxınlaşın amma deyiblər ki asan verir elə sənədi məncə əvvəllər verirdi indi dəyişib dəqiq bilmirəm bir dəqiqə zəhmət olmasa gözləyin alo səsiniz gəlmir zəifdi mən də yaxşı başa düşmürəm nə istəyirsiz sənədi bir də zəng edin bəlkə başqa əməkdaş cavab verə bilər",
    "salam mənim şəxsiyyət vəsiqəmin müddəti keçib onu dəyişmək üçün nə etməliyəm salam buyurun sizə deyiblər biz dəyişirik deyəsən amma indi olmur elə şey bizdə yəni məlumat yoxdur məndə sənəd lazımdı amma nə sənədi bilmirəm dəqiq belə deyiblər sadəcə şəxsiyyət vəsiqəsi üçün adı dəyişib deyiblər onda notarius baxar deyəsən bizdə olmur sənəd vermə proseduru mənlik deyil zəhmət olmasa sonra zəng edin başqa əməkdaş bilər",
    "salam mən ev üçün qeydiyyat arayışı almaq istəyirdim deyiblər asan xidmətdən alınır xanım o qeydiyyatla bağlı sənədlər biz vermirik məncə icra hakimiyyəti baxır ona yox mən konkret soruşuram siz vermirsinizsə deyin mən gedim başqa yerə baxın google yazın görün hardan verilir bizdə olmur bu gün o sənəd deyəsən çıxarılmır sistemdə problem var başqa sualınız varsa buyurun",
    "alo salam mən mehkum olmamağım barədə arayış almaq istəyirəm çünki xaricə sənəd təqdim etməliyəm onu haradan ala bilərəm salam xanım dəqiq deyə bilmərəm vallah bizdə olmur məncə ədliyyə baxır o sənədlərə bəlkə ora gedəsiniz amma bu asan xidmət deyilmi siz verməlisiz axı məncə əvvəllər verilirdi indi necədi bilmirəm zəng edin soruşun ya başqa əməkdaşa yönləndirilsin bir dəqiqə alo səs kəsilir sonra zəng edin yaxşı",
    "salam mən uşağın doğum haqqında şəhadətnaməsini almaq istəyirəm onlayn müraciət etmişdim amma alınmadı onu necə ala bilərəm salam bizdə o sistem işləmir deyəsən indi siz gedin notariusa bəlkə onlar kömək edə bilər onlayn alınmadısa bəlkə sistemdə nasazlıq var mən burada heç nə görə bilmirəm yəni siz deyirsiniz ki asan xidmət vermir məncə əvvəllər verirdi amma indi dəqiq bilmirəm sizə düzgün deyə bilmərəm",
    "alo salam mən xaricə getmək üçün sənədlər toplamalıyam və məhküm olmamağım barədə arayış lazımdır onu necə ala bilərəm salam xanım belə sənədləri biz vermirik deyəsən yəni siz deyirsiniz heç vermirsiniz əvvəllər demişdilər sizdə olurdu indi yox deyəsən dəyişib sistemdə görünmür mənlik deyil siz gedin notariusa ora verə bilər amma dəqiq deyə bilmərəm yaxşı başa düşmədim indi siz verirsiniz ya yox yox yox deyəsən biz vermirik zəhmət olmasa sonra zəng edin bəlkə başqa əməkdaş bilər",
    "salam mənə şəxsiyyət vəsiqəsi üçün qeydiyyat arayışı lazımdır internetdə yazılıb asan xidmətdən götürülür buyurun xanım mən bir şey bilmirəm o arayış bəlkə bələdiyyədən alınır yəni bu barədə məlumatım yoxdur bizdə olmur məncə amma siz deyirsiniz ki internetdə yazılıb bura verilir bəli amma mənlik deyil yəni sistemdə baxmıram mən indi sənədi hardan alım deyirsiniz heç bilmirsiniz zəng edin sabah soruşun mən dəqiq deyə bilmirəm",
    "alo mən vərəsəliklə bağlı sənəd almaq istəyirdim buyurun salam xanım bu barədə heç məlumatım yoxdur vərəsəlik dediniz notarius baxır bizdə olmur sənədi necə ala bilərəm deyirlər bura yaxınlaşmaq lazımdır yox xanım o prosedur bizlik deyil deyəsən məhkəməyə aidiyyatı var zəng edin məhkəməyə bura belə şeylərlə məşğul olmur",
    "salam mənim adım dəyişib şəxsiyyət vəsiqəsini dəyişmək istəyirəm hansı sənədlər lazımdır xanım siz yəqin qeydiyyat yerinə getməlisiniz bizdə olmur sənəd yəni məlumatım yoxdur belə şeylərlə biz baxmırıq əvvəlcə deyirdilər burdan alınır indi yox notarius bəlkə bilir zəhmət olmasa ora gedin mənlik deyil",
    "salam mən xaricə sənəd göndərmək istəyirəm və sənədi təsdiqlətməliyəm buyurun amma xanım bu məsələ ilə məşğul olmuruq belə təsdiq üçün siz başqa yerdə təsdiq alın bəlkə konsulluğa zəng edin bu işlər bizlik deyil",
    "salam mən şəxsiyyət vəsiqəmi itirmişəm və yeni sənəd almaq istəyirəm bu mümkünmü xanım itkin sənədlə bağlı biz məlumat vermirik polisə müraciət edin o sənəd bizlik deyil yəni siz heç nə edə bilmirsiniz yox xanım keçin bölməyə",
    "alo salam mən ailə vəziyyətimlə bağlı sənəd istəyirəm deyiblər asan verir onu xanım mən bilmirəm belə şeylər üçün notarius var deyəsən ailə arayışları biz vermirik valla məlumatım yoxdur zəng edin başqa yerə",
    "salam mənim uşağım üçün doğum şəhadətnaməsini bərpa etdirmək istəyirəm sistemdə görünmür xanım biz belə məlumatlara baxmırıq siz qeydiyyat şöbəsinə getməlisiniz bizdə olmur sənəd mən də buradan nə edə bilərəm",
    "salam mən ev sənədimi təsdiqlətmək istəyirəm yəni alqı-satqı üçün lazımdır buyurun amma xanım bizdə belə şeylər olmur o sənədi siz ya notariusdan alın ya daşınmaz əmlak ofisinə yaxınlaşın asan baxmır bu tip şeylərə",
    "salam mən özümə vasiqə çıxartmaq istəyirəm amma deyiblər bəzi sənədlər lazımdır nə lazımdır deyə bilərsiniz xanım bu məsələyə mən baxmıram ümumiyyətlə nadir işdir deyəsən siz başqa yerə gedin",
    "salam mən müvəqqəti qeydiyyat üçün sənəd almaq istəyirəm çünki evimi dəyişmişəm xanım bu qeydiyyat məsələsi bizlik deyil icra hakimiyyətinə getməlisiniz biz heç nə etmirik belə məsələdə",
    "salam mən şəxsiyyət vəsiqəmi itirmişəm ona görə bank sənədi ala bilmirəm nə etməliyəm buyurun bu barədə heç nə deyə bilmərəm xanım bəlkə banka zəng edin biz burda belə sənədlə kömək edə bilmirik",
    "salam mən mehkum olmamaq arayışı almaq istəyirəm deyiblər sənəd hazırdır amma hələ zəng gəlməyib xanım biz burdan belə məlumatları vermirik gözləyin zəng edərlər mənlik deyil",
    "salam mən ailə vəziyyətimlə bağlı sənəd təqdim etməliyəm əcnəbi vətəndaş üçün bu asan xidmətdə olurmu xanım bilmirəm valla əcnəbilərlə bağlı çox şey dəyişib siz miqrasiya xidmətinə zəng edin bu bizlik deyil",
    "salam şəxsiyyət vəsiqəmi dəyişdirməliyəm çünki evlənmişəm və soyadım dəyişib xanım onu biz etmirik deyəsən evlilik aktı lazım olacaq notarius ya qeydiyyat yerinə yaxınlaşın bizdə olmur",
]

# Create a DataFrame from the synthetic data
df_negatives = pd.DataFrame({
    "Transcription": negative_samples,
    "Label": 0
})

# Append to your original DataFrame
df_combined = pd.concat([df_transcripts, df_negatives], ignore_index=True)

# Optional: Check class balance
print(df_combined['Label'].value_counts())

# Optional: Save to CSV
df_combined.to_csv(".csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv(r".csv")
display(df)
print()
display(df['Transcription'].iloc[0])

In [ ]:
display(df['Transcription'].iloc[110])

## Feature Extraction

In [ ]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: Qadınlar gündə nə qədər protein qəbul etməlidir?',
                'query: Balqabağın ev şəraitində hazırlanması üsulları',
               "passage: Ümumi olaraq, 19-70 yaş arası qadınlar üçün gündəlik protein tələbatı təxminən 46 qramdır. Əgər hamilədirsinizsə və ya idmanla məşğul olursunuzsa, bu miqdar daha yüksək ola bilər.",
               "passage: 1. Sıyıq balqabaq: Balqabağı soyub rəndələyin, azca yağda qızardın, duz və şəkər əlavə edib bişirin. 2. Soğanlı balqabaq qovurması: Balqabaq dilimlərini yağda soğanla birlikdə qovurun, dadına duz vurun."]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

Great — the output looks exactly as expected, and here's how to **interpret the results** of the similarity matrix:

```python
[[86.88, 74.45],
 [75.53, 84.32]]
```

Each row corresponds to a **query**, and each column to a **passage**. So, the matrix shows:

|                   | Passage 1 (protein) | Passage 2 (pumpkin) |
| ----------------- | ------------------- | ------------------- |
| Query 1 (protein) | **86.88**           | 74.45               |
| Query 2 (pumpkin) | 75.53               | **84.32**           |

---

✅ Interpretation:

* **Query 1:** “Qadınlar gündə nə qədər protein qəbul etməlidir?”

  * Highest match is **Passage 1**, with score **86.88** — ✔️ Correct (it's the protein info).
* **Query 2:** “Balqabağın ev şəraitində hazırlanması üsulları”

  * Highest match is **Passage 2**, with score **84.32** — ✔️ Correct (it's the pumpkin recipe).

---

✅ Conclusion:

* The model **correctly matched Azerbaijani queries** with relevant Azerbaijani passages based on semantic meaning.
* This validates that `intfloat/multilingual-e5-large` can perform **cross-language and in-language semantic retrieval** very well — even for **low-resource languages** like Azerbaijani.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Model size: 561M params; Tensor type: F32
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-large')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-large")

# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

# forward pass
output = model(**encoded_input)

In [ ]:
from transformers import BertTokenizer, BertModel

# Model size: 168M params; Tensor type: F32
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

# text = "Replace me by any text you'd like."
text = df['Transcription'].iloc[0]
encoded_input = tokenizer(text, return_tensors='pt')

output = model(**encoded_input)

In [ ]:
output

In [ ]:
embedding = output.pooler_output.detach().numpy().squeeze()
embedding

In [ ]:
import pandas as pd

df = pd.read_csv(r".csv")
display(df)
df.info()

In [ ]:
print(df.duplicated().sum())

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
display(df)
df.info()

In [ ]:
print(df.duplicated().sum())

In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertModel

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {device}")

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased').to(device)
model.eval()

embeddings = []

for text in df['Transcription']:
    # Tokenize and move to device
    encoded = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)

    with torch.no_grad():
        output = model(**encoded)
        cls_embedding = output.pooler_output.detach().cpu().numpy().squeeze()  # move back to CPU for numpy
        embeddings.append(cls_embedding)

X = np.vstack(embeddings)  # Feature matrix
y = df['Label'].values      # Labels

In [ ]:
import pandas as pd
import numpy as np
import os

# Combine embeddings and labels into a DataFrame
df_embeddings = pd.DataFrame(X)
df_embeddings["Label"] = y  # Append label column

# Define output path
output_path = r".csv"

# Ensure directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save to CSV
df_embeddings.to_csv(output_path, index=False)

print(f"✅ Embeddings saved to: {output_path}")

In [ ]:
import pandas as pd

df = pd.read_csv(r".csv")
display(df)
print()
#display(df['Transcription'].iloc[0])

## Model Training

In [ ]:
import pandas as pd

df = pd.read_csv(r".csv")
#display(df)
print(df.duplicated().sum())

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load your dataset
df = pd.read_csv(r".csv")

# Features and labels
X = df.drop(columns=["Label"]).values
y = df["Label"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Define models to evaluate
models = {
    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=1_000, random_state=42))
    ]),
    "SVM (RBF)": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(random_state=42))
    ]),
    "K-NN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier())
    ]),
    # RandomForest, GradientBoosting and GaussianNB often work fine without scaling
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Naive Bayes": GaussianNB()
}

# Train and evaluate each model
for name, model in models.items():
    print(f"\n🔍 Evaluating: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    print(f"✅ Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))

In [ ]:
import joblib

# Access your trained model from the pipeline
knn_model = models["K-NN"]

# Save to file
joblib.dump(knn_model, "/knn_model.pkl")
print("✅ K-NN model saved.")

## Inference

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
import joblib
import numpy as np

# === Load tokenizer and model ===
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased')
model.eval()

# === Load your trained K-NN model pipeline ===
knn_model = joblib.load("../knn_model.pkl")

# === Your custom input text ===
input_text = "alo salam mən sənəd almaq istəyirəm xaricə getmək üçün məhküm olmamağım barədə"

# === Step 1: Lowercase (same as training) ===
input_text = input_text.lower()

# === Step 2: Generate BERT [CLS] embedding ===
encoded = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True)
with torch.no_grad():
    output = model(**encoded)
    embedding = output.pooler_output.detach().numpy().squeeze()

# === Step 3: Predict using your trained model ===
predicted_label = knn_model.predict([embedding])[0]

print(f"🔍 Prediction: {'Yaxşı cavab' if predicted_label == 1 else 'Pis cavab'}")

In [ ]:
import os
import uuid
import torch
import shutil
import numpy as np
import torchaudio
import joblib
from denoiser import pretrained
from denoiser.dsp import convert_audio
from stable_whisper import load_model as load_sw_model
from transformers import BertTokenizer, BertModel
import gradio as gr

# ========== Device Setup ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== Denoising Model ==========
denoise_model = pretrained.dns64().to(device)
DEBUG_DIR = "debug/"
os.makedirs(DEBUG_DIR, exist_ok=True)

def denoise_audio(audio_path):
    wav, sr = torchaudio.load(audio_path)
    wav = convert_audio(wav, sr, denoise_model.sample_rate, denoise_model.chin)
    with torch.no_grad():
        enhanced = denoise_model(wav.to(device))
    enhanced = enhanced.squeeze(0).cpu()
    out_path = os.path.join(DEBUG_DIR, f"denoised_{uuid.uuid4().hex}.wav")
    torchaudio.save(out_path, enhanced, denoise_model.sample_rate)
    return out_path

# ========== Whisper Model ==========
sw_model = load_sw_model("large-v3", device=device)

# ========== BERT + K-NN Setup ==========
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")
bert_model.eval()

knn_model = joblib.load("../knn_model.pkl")

def classify_transcription(text):
    text = text.lower()
    encoded = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    encoded = {k: v.to(device) for k, v in encoded.items()} # work on both GPU and CPU.
    with torch.no_grad():
        output = bert_model(**encoded)
        embedding = output.pooler_output.detach().numpy().squeeze()
    
    encoded = {k: v.to(device) for k, v in encoded.items()}
    with torch.no_grad():
        output = bert_model(**encoded)

    
    prediction = knn_model.predict([embedding])[0]
    label = "Yaxşı cavab ✅" if prediction == 1 else "Pis cavab ❌"
    return label

# ========== Full Processing Pipeline ==========
def process_audio_and_classify(audio_path):
    # 1. Save original
    raw_copy = os.path.join(DEBUG_DIR, f"original_{uuid.uuid4().hex}.wav")
    shutil.copy(audio_path, raw_copy)

    # 2. Denoise
    denoised_path = denoise_audio(audio_path)

    # 3. Transcribe
    result = sw_model.transcribe(denoised_path, language="azerbaijani", word_timestamps=False)
    full_text = result.text.strip()

    # 4. Classify
    label = classify_transcription(full_text)

    # 5. Build HTML output
    html = f"""
    <h3>🔊 Denoised Audio</h3>
    <audio controls src='{denoised_path}' style='width:100%; margin-bottom:16px;'></audio>
    <h3>📄 Transcription</h3>
    <div style='white-space: pre-wrap; border:1px solid #ccc; padding:8px;'>{full_text}</div>
    <h3>🤖 Model Prediction</h3>
    <div style='font-size: 1.2em; font-weight: bold; color: {"green" if "Yaxşı" in label else "red"}'>{label}</div>
    """
    return html

# ========== Gradio UI ==========
with gr.Blocks() as demo:
    gr.Markdown("## PoC: Evaluating Call Center Operator Performance via Call Analysis")
    audio_input = gr.Audio(type="filepath", label="Upload WAV audio")
    output_html = gr.HTML()
    run_button = gr.Button("Process and Classify")

    run_button.click(
        fn=process_audio_and_classify,
        inputs=audio_input,
        outputs=output_html
    )

demo.launch()

## 3CX Demo

In [ ]:
import os
import uuid
import torch
import shutil
import numpy as np
import torchaudio
import joblib
from denoiser import pretrained
from denoiser.dsp import convert_audio
from stable_whisper import load_model as load_sw_model
from transformers import BertTokenizer, BertModel
import gradio as gr
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import threading
import time
import webbrowser

import os
os.environ["LOKY_MAX_CPU_COUNT"] = str(os.cpu_count())

# ========== Device Setup ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ========== Folders Setup ==========
DEBUG_DIR = "debug/"
os.makedirs(DEBUG_DIR, exist_ok=True)
RECORDER_DIR = "records"
os.makedirs(RECORDER_DIR, exist_ok=True)
PROCESSED_DIR = os.path.join(RECORDER_DIR, "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

# ========== Denoising Model ==========
denoise_model = pretrained.dns64().to(device)
def denoise_audio(audio_path):
    wav, sr = torchaudio.load(audio_path)
    wav = convert_audio(wav, sr, denoise_model.sample_rate, denoise_model.chin)
    with torch.no_grad():
        enhanced = denoise_model(wav.to(device))
    enhanced = enhanced.squeeze(0).cpu()
    out_path = os.path.join(DEBUG_DIR, f"denoised_{uuid.uuid4().hex}.wav")
    torchaudio.save(out_path, enhanced, denoise_model.sample_rate)
    return out_path

# ========== Whisper Model ==========
sw_model = load_sw_model("large-v3", device=device)

# ========== BERT + K-NN Setup ==========
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
bert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")
bert_model.eval()
bert_model.to(device)

knn_model = joblib.load("C:/Pasha-PoC/knn_model.pkl")

def classify_transcription(text):
    text = text.lower()
    encoded = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        output = bert_model(**encoded.to(device))
        embedding = output.pooler_output.detach().cpu().numpy().squeeze()
    prediction = knn_model.predict([embedding])[0]
    label = "Yaxşı cavab ✅" if prediction == 1 else "Pis cavab ❌"
    return label

# ========== Full Processing Pipeline ==========
def process_audio_and_classify(audio_path):
    # 1. Save original
    raw_copy = os.path.join(DEBUG_DIR, f"original_{uuid.uuid4().hex}.wav")
    shutil.copy(audio_path, raw_copy)

    # 2. Denoise
    denoised_path = denoise_audio(audio_path)

    # 3. Transcribe
    result = sw_model.transcribe(denoised_path, language="azerbaijani", word_timestamps=False)
    full_text = result.text.strip()

    # 4. Classify
    label = classify_transcription(full_text)

    # 5. Build HTML output
    html = f"""
    <h3>🔊 Denoised Audio</h3>
    <audio controls src='{denoised_path}' style='width:100%; margin-bottom:16px;'></audio>
    <h3>📄 Transcription</h3>
    <div style='white-space: pre-wrap; border:1px solid #ccc; padding:8px;'>{full_text}</div>
    <h3>🤖 Model Prediction</h3>
    <div style='font-size: 1.2em; font-weight: bold; color: {"green" if "Yaxşı" in label else "red"}'>{label}</div>
    """
    return html

# === File Watcher Setup ===
class NewWavHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.endswith(".wav"):
            return
        if "processed" in os.path.normpath(event.src_path).split(os.sep):
            return  # ✅ Skip files inside any "processed" folder
    
        print(f"[Watcher] Detected new file: {event.src_path}")
        
        try:
            # Run full processing
            result_html = process_audio_and_classify(event.src_path)
            print("[Watcher] Processing complete.")

            # Save result to HTML
            result_filename = f"result_{uuid.uuid4().hex}.html"
            result_path = os.path.join(DEBUG_DIR, result_filename)
            with open(result_path, "w", encoding="utf-8") as f:
                f.write(result_html)
            print(f"[Watcher] Result saved to: {result_path}")

            # Open in browser
            webbrowser.open(f"file://{os.path.abspath(result_path)}")

            # Move original file to processed/
            relative_path = os.path.relpath(event.src_path, RECORDER_DIR)
            processed_path = os.path.join(PROCESSED_DIR, relative_path)

            os.makedirs(os.path.dirname(processed_path), exist_ok=True)
            shutil.move(event.src_path, processed_path)
            print(f"[Watcher] Moved file to: {processed_path}")

        except Exception as e:
            print(f"[Watcher] Error processing file: {e}")

def start_file_watcher():
    observer = Observer()
    event_handler = NewWavHandler()
    observer.schedule(event_handler, path=RECORDER_DIR, recursive=True)
    observer.start()
    print("[Watcher] Monitoring started.")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()

# Start the watcher in a background thread
watcher_thread = threading.Thread(target=start_file_watcher, daemon=True)
watcher_thread.start()

# ========== Gradio UI ==========
with gr.Blocks() as demo:
    gr.Markdown("## PoC: Evaluating Call Center Operator Performance via Call Analysis")
    audio_input = gr.Audio(type="filepath", label="Upload WAV audio")
    output_html = gr.HTML()
    run_button = gr.Button("Process and Classify")

    run_button.click(
        fn=process_audio_and_classify,
        inputs=audio_input,
        outputs=output_html
    )

demo.launch()

## Summarization

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """

"""

model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to("cuda")

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"].to("cuda")

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
    # early_stopping=True  # Optional: stop when EOS token is reached
    # length_penalty=
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

**For input**: "*Salam! Xoş gününüz olsun, “X Bank” müştəri xidmətinə zəng etdiyiniz üçün təşəkkür edirik. Mən Ayseləm. 
Sizə necə kömək edə bilərəm? Salam, Aysel xanım. Mən kartımın balansını öyrənmək istəyirəm. Əlbəttə, sizə kömək etmək üçün 
əvvəlcə şəxsiyyətinizi təsdiqləməliyəm. Zəhmət olmasa, adınızı, soyadınızı və doğum tarixinizi qeyd edin. 
Adım Kamran Məmmədov, 12 iyun 1987-ci il. Təşəkkür edirəm, Kamran bəy. Sistemə baxıram... 
Bəli, sizi tapdım. Hal-hazırda kartınızın balansı 524 manat 80 qəpik təşkil edir. Çox sağ olun. 
Bir sualım da var. Kartımda hər ay niyə 2 manat tutulur? Bu kartın aylıq xidmət haqqıdır. 
Əgər istəsəniz, komissiyasız kart növü ilə əvəz edə bilərik. Bəli, maraqlıdır. 
Zəhmət olmasa, ətraflı məlumat verin. Əlbəttə! Yeni kartla bağlı sizə məlumat göndərəcəyik və filialımıza yaxınlaşaraq 
dəyişiklik edə bilərsiniz. Oldu, təşəkkür edirəm. Sizə kömək edə bildiyim üçün məmnunam. Gözəl günlər arzulayıram!*"

**Output is**: "*X Bankın müştəri xidmətinə zəng etdiyiniz üçün təşəkkür edirik.*"

**For input**: "*Rusiya Prezidenti Putin sülh danışıqları üçün İstanbula getməyib. 
Onu Ukrayna ilə danışıqlarda köməkçisi təmsil edir, Kreml açıqlayıb.
Putin İstanbula gəlsəydi, ABŞ Prezidenti Donald Trump da bu görüşə qoşulacağını demişdi.
Mayın 11-də Putin özü Zelenski ilə sülh danışıqlarına çağırış etmişdi.
Zelenski də İstanbulda onu şəxsən gözləyəcəyini demişdi.*"

**Output is**: "*Rusiya Prezidenti Vladimir Putin Ukrayna prezidenti Volodimir Zelenski ilə sülh danışıqları üçün İstanbula gəlməyib.*"

---

### **Summary Conclusion**

The `csebuetnlp/mT5_multilingual_XLSum` model demonstrates strong baseline performance for Azerbaijani text summarization, especially in the context of news articles, as evidenced by accurate summaries of political news input. However, its performance on domain-specific data—such as bank call center conversations—shows limitations due to the conversational and transactional nature of the content, which the model wasn't explicitly trained on.

This model serves as a **good starting point**, but for effective deployment in the banking customer service domain, **fine-tuning is essential**. A fine-tuning strategy can begin with the **"LocalDoc/summarization\_azerbaijan"** dataset to bridge the domain gap, followed by incorporation of **bank-provided audio recordings**. These recordings can be transcribed using **Whisper-Large-V3**, post-processed and cleaned with a **large language model (LLM)**, **manually reviewed a sample set** to *validate the LLM corrections* and then used to fine-tune the model for **dialogue-based summarization**.

These transcriptions can be used in two complementary ways:

* **As-is (noisy STT output):** Fine-tune the summarizer to handle real-world, imperfect input, improving robustness.
* **After correction (cleaned by LLM):** Fine-tune on clean data for higher precision and clarity in summaries.

Alternatively, **Whisper itself can be fine-tuned** (if needed and feasible) on domain-specific audio to produce **higher-quality transcriptions**, reducing (or eliminating) the need for post-processing before summarization fine-tuning.

This approach ensures adaptability to both **raw conversational input** and **high-quality cleaned text**, supporting a production-grade summarization system for banking customer interactions in Azerbaijani. This pipeline ensures the summarization model will learn the structure, terminology, and conversational nuances specific to Azerbaijani banking customer service contexts.